# Data Portal adaptation demo

This demo implements an analytical workflow that integrates...
* Climate-hazard probability datasets linked from Google Earth Engine
* Interactivity
* Data outputs compatible with adaptation reporting under the CRF

What to notice:
* Current and future hazard magnitudes and probabilities are displayed in the tables, the graph, and the map
* Moving the location marker or the sliders causes all data outputs to update
* The demo has three sections; be sure to scroll all the way to the right

----------------------------------


In [1]:
%matplotlib widget
import matplotlib.pyplot as plt

In [2]:
# GEE authentication using service account key stored in local directory
import ee
#service_account = 'climate-hazard-demo@data-portal-adaptation.iam.gserviceaccount.com'
#credentials = ee.ServiceAccountCredentials(service_account, 'data-portal-adaptation.json')
ee.Initialize()

In [3]:
import geemap
from ipyleaflet import CircleMarker, Polygon, LayersControl, DrawControl, ZoomControl
from ipywidgets import interact, interactive, interact_manual, Layout
import ipywidgets as widgets
import asyncio
import numpy as np

In [4]:
# Create decorator to prevent events triggers while slider is being moved
# From https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html
class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def start(self):
        self._task = asyncio.ensure_future(self._job())

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ Decorator that will postpone a function's
        execution until after `wait` seconds
        have elapsed since the last time it was invoked. """
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
            timer.start()
        return debounced
    return decorator

In [5]:
# Load data from GEE
MAPprobs = ee.ImageCollection("users/tedwongwri/MAP_probs/ehe_MAPprob_prime01")
NC_MAPprobs = ee.ImageCollection("users/tedwongwri/MAP_probs/noncumul_ehe_MAPprob_prime01")

In [6]:
initial_threshold = 30
currentYear = 2022
futureYear = 2050
prev_threshold = 30
prev_futureYear = 2050
threshold_value = initial_threshold
initial_coords = [50.85045, 4.34878]  # Brussels
current_geom = ee.Geometry.Point(initial_coords)
current_location_marker = None

In [7]:
futureyear_slider = widgets.IntSlider(
    value = futureYear,
    min = currentYear + 1,
    max = 2080,
    step = 1,
    description = 'Future year',
    continuous_update = False
)

threshold_slider = widgets.IntSlider(
    value=initial_threshold,
    min=5,
    max=50,
    step=5,
    description='EHE ≥ ___',
    continuous_update=False
)

prob_display = widgets.HTML()
ev_display = widgets.HTML()
debug_display = widgets.HTML()

In [8]:
currentprobs = MAPprobs.filterMetadata('year', 'equals', currentYear).first()
futureprobs = MAPprobs.filterMetadata('year', 'equals', futureYear).first()

In [9]:
Map = geemap.Map(center=initial_coords, zoom=3)
Map.add_basemap('HYBRID')

Map.clear_controls()
layer_control = LayersControl(position='topright')
Map.add_control(layer_control)
Map.add_control(ZoomControl(position='bottomright'))
draw_control = DrawControl(circle={}, polyline={}, rectangle={}, position='topleft')
Map.add_control(draw_control)


current_location_marker = CircleMarker(location=initial_coords, name="location of interest")
Map.add_layer(current_location_marker)

vizParamsC = {
  'bands': ['threshold_' + str(threshold_value)],
  'min': 0,
  'max': 1,
  'palette': ['000000', '909000'],
  'opacity': 0.5
}
vizParamsF = {
  'bands': ['threshold_' + str(threshold_value)],
  'min': 0,
  'max': 1,
  'palette': ['000000', 'ff0000'],
  'opacity': 0.5
}
Map.addLayer(currentprobs, vizParamsC, str(currentYear) + ' Prob(EHE ≥ ' + str(threshold_value) + ')')
Map.addLayer(futureprobs, vizParamsF, str(futureYear) + ' Prob(EHE ≥ ' + str(threshold_value) + ')')

In [10]:
def expected_value(probs, mags):
    np_probs = np.array(probs)
    np_mags = np.array(mags)
    return np.dot(np_probs, np_mags) / np_probs.size

def get_ev_ehe(year):
    # Plot expected value of EHE for given year
    current_NC_probs = NC_MAPprobs.filterMetadata('year', 'equals', year).first().rename([str((i+1)*5) for i in range(25)] )
    current_NC_data = current_NC_probs.reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo()
    dataprobs = [float(i) for i in current_NC_data.values()]
    datamags = [float(i) for i in list(current_NC_data.keys())]
    return expected_value(dataprobs, datamags)

In [11]:
def renameBands(img):
    # MAPprob files have bandnames like "threshold_5"
    return img.rename(['05', '10', '15', '20', '25', '30', '35', '40', '45', '50'] )

def prob(threshold, year):
# Returns prob of given EHE threshold for given year, as float betw 0 and 1 inclusive
# Threshold: 5-50 by fives; Year: 2022-2080 by ones    
    data = renameBands(MAPprobs.filterMetadata('year', 'equals', year).first()).reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo()
    return data[['0',''][threshold > 5] + str(threshold)]

def prob_interp(magnitude, year):
    # Finds prob by finding magnitude values in the data that are below and above given magnitude
    # and treating prob as linear between the two.
    data = NC_MAPprobs.filterMetadata('year', 'equals', year).first().rename([str((i+1)*5) for i in range(25)]).reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo()
    probs = np.array([float(i) for i in data.values()])
    mags = np.array([float(i) for i in data.keys()])
    lte = np.nonzero(mags <= magnitude)[0]
    lte_idx = -1
    if lte.size == 0:
        lte = 0
    else:
        lte_idx = lte[-1]
        lte = mags[lte_idx]
    if lte_idx < mags.size - 2:
        slope = (magnitude - lte) / (mags[lte_idx + 1] - lte)
        if lte_idx >=0:
            return probs[lte_idx] + (slope * (probs[lte_idx + 1] - probs[lte_idx]))
        else:
            probs[0]
    else:
        return probs[-1]

def update_map():
    global futureprobs
    global futureYear
    global prev_futureYear
    global prev_threshold
    new_visparamsC = {
      'bands': ['threshold_' + str(threshold_value)],
      'min': 0,
      'max': 1,
      'palette': ['000000', '909000'],
      'opacity': 0.5
    }
    new_visparamsF = {
      'bands': ['threshold_' + str(threshold_value)],
      'min': 0,
      'max': 1,
      'palette': ['000000', 'ff0000'],
      'opacity': 0.5
    }
    futureprobs = MAPprobs.filterMetadata('year', 'equals', futureYear).first()
    Map.remove_ee_layer(name=str(currentYear) + ' Prob(EHE ≥ ' + str(prev_threshold) + ')')
    Map.remove_ee_layer(name=str(prev_futureYear) + ' Prob(EHE ≥ ' + str(prev_threshold) + ')')
    Map.add_ee_layer(currentprobs, new_visparamsC, str(currentYear) + ' Prob(EHE ≥ ' + str(threshold_value) + ')')
    Map.add_ee_layer(futureprobs, new_visparamsF, str(futureYear) + ' Prob(EHE ≥ ' + str(threshold_value) + ')')
    prev_threshold = threshold_value
    prev_futureYear = futureYear

def CRFtext(probstring, prob):
    if prob > 0.05:
        return 'high'
    elif prob > 0.005:
        return 'moderate'
    elif prob > 0.0005:
        return 'low'
    else:
        return 'do not know'
    
def changetext(before, after, inv):
    # Assigns label to numerical probability according to Box 4 (p. 44) of
    # https://www.globalcovenantofmayors.org/wp-content/uploads/2019/08/Data-TWG_Reporting-Framework_GUIDENCE-NOTE_FINAL.pdf
    if before == after:
        return 'no change'
    chg = after > before
    if inv:
        chg = not chg
    return ['decrease', 'increase'][chg * 1]
    
def update_evdisplay():
    # Inserts output HTML into evdisplay widget
    # Outputs are expected value of non-cumulative (that is, EHE=x, not EHE≥x) EHE value, current and future

    current_ev = get_ev_ehe(currentYear)
    future_ev = get_ev_ehe(futureYear)
    current_prob = prob_interp(current_ev, currentYear)
    future_prob = prob_interp(future_ev, futureYear)
    evstring_current = '{:.1f} days'.format(current_ev)
    evstring_future = '{:.1f} days'.format(future_ev)
    probstring_current = '{:.1f}%'.format(current_prob * 100)
    probstring_current += ' (' + CRFtext(probstring_current, current_prob) + ')'
    probstring_future = '{:.1f}%'.format(future_prob * 100)
    probstring_future += ' (' + CRFtext(probstring_future, future_prob) + ')'
    newHtml = '<b>What EHE can you expect?</b><table style="width:500px; border: 1px solid black;"><tr style="border-bottom: 2px solid black; text-align: center; font-weight: bold"><td>year</td><td>magnitude</td><td>probability</td><td>magnitude change</td><td>frequency change</td></tr>'
    newHtml += '<tr style="padding: 3px; text-align: center"><td>' + str(currentYear) + '</td><td>' + evstring_current + '</td><td>' + probstring_current + '</td><td rowspan="2">' + changetext(current_ev, future_ev, False) + '</td><td rowspan="2">' + changetext(current_prob, future_prob, False) + '</td></tr>'
    newHtml += '<tr style="padding: 3px; text-align: center"><td>' + str(futureYear) + '</td><td>' + evstring_future + '</td><td>' + probstring_future + '</td></tr></table>'
    ev_display.set_trait('value', newHtml)
    

    
def update_const_mag_display():
    # Inserts output HTML into probdisplay widget
    # Outputs are current and future probability of user-selected EHE threshold value
    current_prob = prob(threshold_value, currentYear)
    future_prob = prob(threshold_value, futureYear)
    probstring_current = '{:.1f}%'.format(current_prob * 100)
    probstring_current += ' (' + CRFtext(probstring_current, current_prob) + ')'
    probstring_future = '{:.1f}%'.format(future_prob * 100)
    probstring_future += ' (' + CRFtext(probstring_future, future_prob) + ')'
    ri_string_current = '{:.1f} years'.format(1.0/current_prob)
    ri_string_future = '{:.1f} years'.format(1.0/future_prob)
    newHtml = '<b>How probable is EHE ≥ ' + str(threshold_value) + '?</b><table style="width:500px; border: 1px solid black;"><tr style="border-bottom: 2px solid black; text-align: center; font-weight: bold"><td>year</td><td>probability</td><td>recurrence interval</td><td>frequency change</td></tr>'
    newHtml += '<tr style="padding: 5px; text-align: center"><td>' + str(currentYear) + '</td><td style="text-align: center">' + probstring_current + '</td><td>' + ri_string_current + '</td><td rowspan="2">' + changetext(current_prob, future_prob, False) + '</td></tr>'
    newHtml += '<tr style="padding: 5px; text-align: center"><td>' + str(futureYear) + '</td><td style="text-align: center">' + probstring_future + '</td><td>' + ri_string_future + '</td></tr></table>'
    prob_display.set_trait('value', newHtml)

def update_probplot():
    # Replots plots of probability vs EHE threshold value
    probplot.canvas.draw()
    probplot.canvas.flush_events()
    global current_plot
    global future_plot
    global plt
    if current_plot:
        dummy = current_plot.pop(0)
        dummy.remove()
    if future_plot:
        dummy = future_plot.pop(0)
        dummy.remove()
    currentprobs = MAPprobs.filterMetadata('year', 'equals', currentYear).first()
    current_data = renameBands(currentprobs).reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo()
    current_plot = plt.plot([str(i) + 'd' for i in current_data.keys()], current_data.values(), label=str(currentYear), color='green')
    futureprobs = MAPprobs.filterMetadata('year', 'equals', futureYear).first()
    future_data = renameBands(futureprobs).reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo()
    future_plot = plt.plot([str(i) + 'd' for i in future_data.keys()], future_data.values(), label=str(futureYear), color='lime')
    plt.legend()

In [12]:
@debounce(0.5)
def threshold_update(e):
    global threshold_value
    threshold_value = threshold_slider.value
    update_const_mag_display()
    update_map()
@debounce(0.5)
def futureyear_update(e):
    global futureYear
    futureYear = futureyear_slider.value
    update_probplot()
    update_const_mag_display()
    update_evdisplay()
    update_map()
@debounce(0.5)
def marker_update(e):
    update_probplot()
    update_const_mag_display()
    update_evdisplay()
@debounce(0.5)
def drawcontrol_update(e):
    def latlon_to_lonlat(coords):
        if type(coords[0]) == float:
            return [coords[1], coords[0]]
        else:
            return [[[i[1],i[0]] for i in j] for j in coords]
        
    global current_geom
    global current_location_marker
    Map.remove_layer(current_location_marker)
    draw_control.clear()
    new_geom = draw_control.last_draw['geometry']
    if new_geom['type'] == 'Point':
        current_geom = ee.Geometry.Point(new_geom['coordinates'])
        current_location_marker = CircleMarker(location=latlon_to_lonlat(new_geom['coordinates']), name="location of interest")
        Map.add_layer(current_location_marker)
    else:
        current_geom = ee.Geometry.Polygon(new_geom['coordinates'])
        current_location_marker = Polygon(locations=latlon_to_lonlat(new_geom['coordinates']), name="location of interest")
        Map.add_layer(current_location_marker)
    update_probplot()
    update_const_mag_display()
    update_evdisplay()

threshold_slider.observe(threshold_update)
futureyear_slider.observe(futureyear_update)
draw_control.observe(drawcontrol_update)

In [13]:
plt.ioff()
probplot = plt.figure(figsize=(5,4))
current_plot = None
future_plot = None
plt.xlabel('EHE ≥ threshold')
plt.ylabel('probability')
#plt.legend()

plt.ion();

In [14]:
update_probplot()
update_const_mag_display()
update_evdisplay()

In [15]:
map_box = widgets.Box([Map], layout=Layout(width='500px', height='600px'))
hbar = widgets.HTML(value='<br/><div style="height:6px; width: 500px; background-color: #9090c0"></div>')
ev_box = widgets.VBox([futureyear_slider, ev_display], layout=Layout(align_items='flex-start'))
prob_box = widgets.VBox([threshold_slider, prob_display], layout=Layout(align_items='flex-start'))
middle_box = widgets.VBox([debug_display, ev_box, hbar, prob_box])
items = [map_box, middle_box, probplot.canvas]
gridbox = widgets.GridBox(items, layout=Layout(grid_template_columns="repeat(3, 600px)"))
gridbox

GridBox(children=(Box(children=(Map(center=[50.85045, 4.34878], controls=(LayersControl(options=['position'], …